Karen J Yang
October 22, 2016

# Logistic Regression Model to Predict Homelessness Risk

Problem 1: 

Detect or even predict who in the community is at risk of becoming homeless before they end up on the streets, and raplidly connect them with services providers who can help. Remember that many families and individuals become homeless from sudden loss of a job, sexual or physical abuse, addiction, medical debt, mental illness, or unpaid utility bills.

General Observation 1:
Sample dataset was composed of multiple excel sheets that had data from multiple agencies that collected different types of data from their own organization. User IDs were not consistent across the sheets and there were missing data. Datasets ranged in number of observations from 300 - 500.  There were no data in the dataset for people who were non-homeless, which made the target(outcome) variable lack variation.

General Observation 2:
After collaborating with St. Patrick's staff, I learned that the homeless figure nationally is about half a million. This comes to a rough ballpark estimate of 2% homeless nationally. This includes folks that are "couch surfers" as well as those counted in homeless shelters and on the street. Roughly 98% of the population is non-homeless. Data is skewed so fallout may happen during the calculations due to insufficient data.

General Observation 3:
The current sample dataset is unusable for the project but it is still possible to create code for a future feasible dataset. A mock dataset was created to build prediction models. 

In [1]:
# read dataset2.csv
import pandas as pd
import numpy as np
import statsmodels.api as sm

path = '/Users/karenyang/Desktop/dataset2.csv'
df = pd.read_csv(path)  # create dataframe

In [2]:
len(df)  # 1000 observations

1000

In [3]:
# examine the class distribution on the outcome variable
# Counts match to national estimate of 2% homeless(coded as 1) and 98% non-homeless(coded as 0)
df.risk.value_counts()

0    980
1     20
Name: risk, dtype: int64

In [7]:
# examine the class distribution on the predictors
df.sudden_job_loss.value_counts()  # sudden job loss==1, 0 otherwise

1    501
0    499
Name: sudden_job_loss, dtype: int64

In [8]:
# examine the class distribution on the predictors
df.sex_phys_abuse.value_counts()  # sex or physical abuse==1, 0 otherwise

0    825
1    175
Name: sex_phys_abuse, dtype: int64

In [9]:
df.addiction.value_counts() # addiction==1, 0 otherwise

0    900
1    100
Name: addiction, dtype: int64

In [10]:
df.med_debt.value_counts() # medical debt==1, 0 otherwise

0    900
1    100
Name: med_debt, dtype: int64

In [11]:
df.mental_ill.value_counts() # mental illness==1, 0 otherwise

0    900
1    100
Name: mental_ill, dtype: int64

In [12]:
df.unpaid_utility_bills.value_counts() # unpaid utility bills==1, 0 otherwise

0    900
1    100
Name: unpaid_utility_bills, dtype: int64

In [14]:
# Create formula for regression. Build model using glm() function which is part of the formula submodule (statsmodel)
formula = 'risk ~ sudden_job_loss+sex_phys_abuse+addiction+med_debt+mental_ill+unpaid_utility_bills'

In [15]:
# glm() fits generalized linear models, a class of model that includes logistic regression
import statsmodels.formula.api as smf
model = smf.glm(formula=formula, data=df, family=sm.families.Binomial())
result = model.fit()

In [16]:
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   risk   No. Observations:                 1000
Model:                            GLM   Df Residuals:                      993
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -79.405
Date:                Sat, 22 Oct 2016   Deviance:                       158.81
Time:                        16:36:37   Pearson chi2:                     430.
No. Iterations:                    27                                         
                           coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept               -3.1023      0.297    -10.459      0.000        -3.684    -2.521
sudden_job_loss       

In [17]:
#print("Coefficients")
print(result.params)

Intercept               -3.102287
sudden_job_loss        -25.497909
sex_phys_abuse           1.023986
addiction               -1.066563
med_debt               -24.114835
mental_ill              -0.353743
unpaid_utility_bills    -0.564323
dtype: float64


In [18]:
#print("p-Values")
print(result.pvalues)

Intercept               1.338693e-25
sudden_job_loss         9.995129e-01
sex_phys_abuse          4.531242e-02
addiction               3.048437e-01
med_debt                9.997735e-01
mental_ill              6.446779e-01
unpaid_utility_bills    4.592105e-01
dtype: float64


In [19]:
#print("Dependent Variables")
print(result.model.endog_names)

risk


In [20]:
predictions=result.predict()
# Show the first 10 probabilities
print(predictions[0:10])

[  4.30129999e-02   1.06200227e-12   1.11223756e-01   1.52343576e-02
   2.49258008e-02   4.30129999e-02   1.52343576e-02   4.30129999e-02
   1.11223756e-01   1.07441346e-02]


In [21]:
# Convert predicted probabilites into class labels so that we can make predictions
predictions_nominal = [1 if x > 0.5 else 0 for x in predictions]

In [22]:
# Determine how many were correctly or incorrectly classified
# The on-diagonal numbers reflect correct preditions while the off-diagonal numbers reflect incorrect predictions
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df['risk'], predictions_nominal))

[[980   0]
 [ 20   0]]


In [23]:
print(classification_report(df['risk'], predictions_nominal, digits=3))

             precision    recall  f1-score   support

          0      0.980     1.000     0.990       980
          1      0.000     0.000     0.000        20

avg / total      0.960     0.980     0.970      1000



/Applications/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
# Trained and tested on the same dataset.
# Training error rate is typically over-optimistic since it underestimates the test error rate.
# Next step is to split dataset and train to build model on 1st set and then test model off of the 2nd dataset.

In [25]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(df, test_size = 0.2)
x_train = train[['sudden_job_loss', 'sex_phys_abuse', 'addiction', 'med_debt', 'mental_ill', 'unpaid_utility_bills']]
y_train = train.risk

x_test = test[['sudden_job_loss', 'sex_phys_abuse', 'addiction', 'med_debt', 'mental_ill', 'unpaid_utility_bills']]
y_test = test.risk

In [26]:
# Specify the formula
# Build model using glm() function which is part of the formula submodule (statsmodel)
formula1 = 'risk ~ sudden_job_loss+sex_phys_abuse+addiction+med_debt+mental_ill+unpaid_utility_bills'

In [27]:
# train model on 1st dataset
model1 = smf.glm(formula=formula1, data=train, family=sm.families.Binomial())
result1 = model1.fit()

In [28]:
# Use model to predict homelessness risk, using test set data
predictions1 = result1.predict(x_test)

In [29]:
# Convert predicted probabilites into class labels so that we can make predictions
predictions_nominal_test = [1 if x > 0.5 else 0 for x in predictions1]

In [30]:
print(classification_report(y_test, predictions_nominal_test, digits=3))


             precision    recall  f1-score   support

          0      0.980     1.000     0.990       196
          1      0.000     0.000     0.000         4

avg / total      0.960     0.980     0.970       200



/Applications/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
# Test error rate is 1 - recall, which is zero. Data is too skewed for test error rate to be a good metric. 
# Purpose of mock dataset is to build the frameworks for the models. Results will be different for the actual dataset.

In [32]:
# Refit a logistic model using only 2 variables, sudden job loss and medical debt, for illustration only.
# When using a real dataset, use top predictors to re-run models to see if prediction results are improved.
train2 = train[['sudden_job_loss', 'med_debt', 'risk']]
x_train2 = train[['sudden_job_loss', 'med_debt']]
y_train2 = train.risk

x_test = test[['sudden_job_loss', 'med_debt']]
y_test = test.risk

# Build model using glm() function which is part of the formula submodule (statsmodel)
formula2 = 'risk ~ sudden_job_loss+med_debt'

model2 = smf.glm(formula=formula2, data=train2, family=sm.families.Binomial())
result2 = model2.fit()
predictions2=result2.predict(x_test)
# Show the first 10 probabilities
print(predictions2[0:10])

[  4.49438202e-02   4.67798616e-13   4.67798616e-13   4.49438202e-02
   4.67798616e-13   4.67798616e-13   4.49438202e-02   4.67798616e-13
   4.49438202e-02   4.49438202e-02]


In [33]:
predictions_nominal_test2 = [1 if x > 0.5 else 0 for x in predictions2]
print(classification_report(y_test, predictions_nominal_test2, digits=3))

             precision    recall  f1-score   support

          0      0.980     1.000     0.990       196
          1      0.000     0.000     0.000         4

avg / total      0.960     0.980     0.970       200



/Applications/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [37]:
# predicting homeless or non-homeless with new data on sudden job loss and medical debt variables only 
y_pred = result2.predict(pd.DataFrame([[0,0],[1,1]], columns = ["sudden_job_loss","med_debt"]))

In [38]:
print(y_pred)  # predicted outcomes for new data turn out to be 0, which is non-homeless

[  4.49438202e-02   1.31853313e-23]


Point 1:

Null accuracy is predicting off of the outcome variable based on its proportion. Without knowing the factors for homelessness, if one were to predict person is non-homeless 100% of the time, the error rate would be roughly 2%. Still, it is worth pursuing prevention steps and identifying the on onset of homelessness prior to its happening since it is a substantive issue that can affect as many as 0.5 million lives per year.

Point 2:

Future data collection efforts needs to focus on standardizing the features that are used across organizations to better track people vulnerable to homelessness and to offer a better understanding of these factors.

Point 3:

The roughly 2% estimate of homelessness is relatively small, suggesting human resourcefulness and ingenuity to solve their own housing problems by "couch surfing", "managing", or taking other measures.

Point 4:

Risk factors for homelessness can be used as a prevention measure to help improve lives, provide shelter, and to create stability within the community. With an estimated 47% of jobs predicted to be taken over by computers over the next 10 to 20 years, local leaders need to create a massive housing plan, possibly rehabbing vacant and abandoned city-owned properties into readily made available housing units for temporary or permanent housing for a future homeless population that will most likely exceed 2%. 
    
